# Creating and Modifying Data and Databases in SQL

Now it is time to go back to the SQLite command prompt (boo) and do some hacking on the structure of the databases.

## Opening SQLite Prompt

* Open a terminal next to this Notebook
    * `File -> New -> Terminal`
    * Drag the terminal window tab to the center right of the window to do side-by-side display.
* Navigate to the Week 8 directory. 
    * Use completion by type part of the directory name and hitting tab
    * Or type it all out and put it in quotes (because of the spaces in the name) 

```
$ cd "data-basics-fall-2019/8 - databases two"
```

* Make a backup of the database by running the command `cp survey.db survey.db.backup`
* Now run the `sqlite3` command and open the `survey.db` file 
```$ sqlite3 survey.db```

* Now you should see the SQLite prompt

```
SQLite version 3.13.0 2016-05-18 10:57:30
Enter ".help" for usage hints.
sqlite>
```

* Run the `.tables` command to ensure everything is loaded up correctly
```
sqlite> .tables
Person   Site     Survey   Visited
sqlite>
```


* You are now running an SQlite shell, which is yet another command line interface...Yay!


### Sprucing up the SQLite3 prompt

* By default the SQLite prompt produces ugly output

* We can better by running the following sqlite (not sql) commands

* Now the ouput will look much nicer

## Modifying Data

So far we have only looked at how to get information out of a database, both because that is more frequent than adding information, and because most other operations only make sense once queries are understood.

Using the Survey databases, we can add, change, and remove records using our other set of commands, INSERT, UPDATE, and DELETE.

The simplest form of INSERT statement lists values in order:

We can also insert values into one table directly from another:

Modifying existing records is done using the UPDATE statement. To do this we tell the database which table we want to update, what we want to change the values to for any or all of the fields, and under what conditions we should update the values.

For example, if we made a mistake when entering the lat and long values of the last INSERT statement above:

Be careful to not forget the WHERE clause or the update statement will modify all of the records in the database.

Deleting records can be a bit trickier, because we have to ensure that the database remains internally consistent. If all we care about is a single table, we can use the DELETE command with a WHERE clause that matches the records we want to discard. For example, once we realize that Frank Danforth didn’t take any measurements, we can remove him from the Person table like this:

But what if we removed Anderson Lake instead? Our Survey table would still contain seven records of measurements he’d taken, but that’s never supposed to happen: Survey.person is a foreign key into the Person table, and all our queries assume there will be a row in the latter matching every value in the former.

This problem is called referential integrity: we need to ensure that all references between tables can always be resolved correctly. One way to do this is to delete all the records that use 'lake' as a foreign key before deleting the record that uses it as a primary key. 

Currently this database isn't designed to enforce integrity constraints. Let's fix that!


## Creating Schemas

The main commands for creating database tables are CREATE TABLE and DROP TABLE. While they are written as two words, they are actually single commands. The first one creates a new table; its arguments are the names and types of the table’s columns. 

For example, run the `.schema` command to see the four CREATE TABLE commands that created the survey database:

This collection of CREATE TABLE statements is the actual SQL code used to create that database. It creates four tables (Person, Site, Visited, and Survey) and specifies various columns with names and data types. We can copy and paste these commands directly to create a new database with the same schema structure. Because the `.schema` command doesn't format things pretty, we will re-write the output below so we can read it better.


Here is a more human friendly formatting style:
```sql
CREATE TABLE Person(
    id text, 
    personal text, 
    family text
);
CREATE TABLE Site(
    name text, 
    lat real, 
    long real
);
CREATE TABLE Visited(
    id integer, 
    site text, 
    dated text
);
CREATE TABLE Survey(
    taken integer, 
    person text, 
    quant text, 
    reading real
);
```

Now you can clearly see this database has four *entities* and each has three or four *properties* (to use the language of relational databses). This is a visual representation of the database.

![Diagram of the survey database](https://swcarpentry.github.io/sql-novice-survey/fig/sql-join-structure.svg)

Currently this database has *implicit* relational structure. The Survey table doesn't explicitly make a connection between the tables, even though they are connected in the data itself. Let's take a look.

* Looking at the `taken` and `person` columns we can see they resemble identifiers or pointers to more complete information. If we look at the Person and Visited tables we can see how the data are connected.

* Looking at this table we can see the `site` column references yet another table. Let's keep following the thread of the data.

* This implicit connection in the data between the various tables is what enables us to do the 

* Without the explict schema defined *integrity constraints* baked into the relational structure of the database, then things like this can happen.

* But there is no Person with an id 'matt'
* Now we have data problems

### Deleting Tables

We can get rid of one of our tables using:

Be very careful when doing this: if you drop the wrong table, hope that the person maintaining the database has a backup, but it’s better not to have to rely on it.

Let's confirm the table has been removed.

Different database systems support different data types for table columns, but most provide the following: 
* INTEGER (a signed integer), 
* REAL (a floating point number), 
* TEXT (a character string), and 
* BLOB (a 'binary large object' such as an image).

Most databases also support Booleans and date/time values; SQLite uses the integers 0 and 1 for the former, and represents the latter as strings. An increasing number of databases also support geographic data types, such as latitude and longitude. Keeping track of what particular systems do or do not offer, and what names they give different data types, is an unending portability headache.

When we create a table, we can specify several kinds of constraints on its columns. For example, a better definition for the Survey and Person tables would be:

Let's break the `CREATE TABLE Survey` statement down.
* First, we create a `taken` column that is an integer number and specify the column can't have null values.
* Second, we create a `person` column that contains text values.
* Third, we create a `quant` column that also contains text values, but also can't have null values.
* Forth, we createa a `reading` column that is a floating point number and can't have null values.
* Fifth, we specify a primary key constraint that is made up of both the `taken` and `quant` values because the combination of these two values will be unique for each row. 
* Sixth, we create a foreign key constraint that creates explicit structural connections between each row in the Survey table with a row in the Person table. This features is what puts the *relational* in relational databases.

By specifying Primary and Foreign Keys we have made the relationships between the tables *explicit* and can be sure that when we enter data into the database (discussed below) we won't enter bad data (like creating a Survey referencing a person whose ID doesn't exist in in the Person table. Both of these mechanisms allow you to express a way of structurally specifying the integrity of the databsae when you create it. This is helpful because now you don't have to write as much error checking code when you are pulling data out! You can rest easy knowing that database system will do data cleaning for you.

![Rest Easy Pig](https://media.giphy.com/media/G5wGy8pDEtAek/giphy.gif)

Exactly what constraints are available and what they’re called depends on which database system we are using.

## Key Points:
- Use INSERT to add data.
- Use UPDATE to modify existing data.
- Use DELETE to remove data.
- Use CREATE and DROP to create and delete tables.
- It is simpler and safer to modify data when every record has a unique primary key.